In [3]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize(project='ee-ana-zonia')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

In [16]:
# region of interest is the entire country
roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").geometry().dissolve()
# select ecoregions to run analysis
ecoregions = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(roi) \
                .map(lambda feature: feature.intersection(roi))

img = ee.Image('projects/ee-ana-zonia/assets/lulc')

## create an empty object to hold stratified points
empty_br = ee.Image().byte().paint(ecoregions, "ecoregion").rename("ecoregion")

In [17]:
## stratify sample points per feature
stratified = empty_br.addBands(ee.Image.pixelLonLat()).stratifiedSample(
      numPoints = 1000,
      classBand = 'ecoregion',
      projection = 'EPSG:4326',
      scale = 30,
      region = ecoregions
)

In [19]:
## sample image using stratified points ('random' is the default column name given by randomColumn)
sample = img.sampleRegions(collection=stratified, properties=['ecoregion', 'random'], scale=30)

In [20]:
task = ee.batch.Export.table.toDrive(
    collection = sample,
    description = 'lulc_sampled',
    folder = 'datasheets'
)

task.start()


In [ ]:
frag = ee.Image('projects/ee-ana-zonia/assets/frag_2020')
land_use = LU_sum.addBands([last_LU, last_fire, num_fires, frag, median_mature])

ecoreg = ee.Image("projects/ee-ana-zonia/assets/ecoregions_br")
indig = ee.Image("projects/ee-ana-zonia/assets/indig")
soil = ee.Image("projects/ee-ana-zonia/assets/soil")
protec = ee.Image("projects/ee-ana-zonia/assets/protec")
biomes = ee.Image("projects/ee-ana-zonia/assets/biome_br")
categorical = ecoreg.addBands([indig, soil, protec, biomes])

yearly_si = ee.Image("projects/ee-ana-zonia/assets/yearly_si")
mean_prec = ee.Image("projects/ee-ana-zonia/assets/mean_prec")
cwd_chave = ee.Image("projects/ee-ana-zonia/assets/cwd_chave").clip(roi)
climate = yearly_si.addBands([mean_prec, cwd_chave])
